# 5. Blind rotation
A blind rotation procdeure is the core of FHEW bootstrapping and other many application of HE supporting non-arithmetic (not $\times$ or +) operations.

It is defined as follows.

**Definition (blind rotation)**
- Input: LWE ciphertext $(\beta, \vec{\alpha})$, blind rotation keys $brk$ under secret $\boldsymbol{z}$, and public polynomial $f$, where $\beta + \left< \vec{\alpha}, \vec{s}\right> = u$
- Output: $RLWE_{\boldsymbol{z}}(f\cdot X^u)$

It is compose of following three steps.
1. Make a encryption of $f\cdot X^{\beta}$, $ACC$. It can easily be done $ACC = (f\cdot X^{\beta}, 0)$.
2. Accumulation: homomophically multiply $X^{\alpha_i s_i}$ to $ACC$ and update it.
3. After accumulation, we get $RLWE_{\boldsymbol{z}}(f\cdot X^{\beta + \left< \vec{\alpha}, \vec{s}\right>}) = RLWE_{\boldsymbol{z}}(f\cdot X^u)$

In [533]:
# Functions from previous lecturenote
import torch
import math

stddev = 3.2
logQ = 27

N = 2**10
Q = 2**logQ

n = 512
q = 2*N

def keygen(dim):
    return torch.randint(2, size = (dim,), dtype=torch.int32)

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def errgen(stddev, N):
    e = torch.round(stddev*torch.randn(N))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,), dtype=torch.int32)

def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] -= a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

root_powers = torch.arange(N//2).to(torch.complex128)
root_powers = torch.exp((1j*math.pi/N)*root_powers)

root_powers_inv = torch.arange(0,-N//2,-1).to(torch.complex128)
root_powers_inv = torch.exp((1j*math.pi/N)*root_powers_inv)

def negacyclic_fft(a, N, Q):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[...,:N//2] + 1j * acomplex[..., N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((b.real, b.imag), dim=-1)
    # Rounding should be more accurate
    # a += 0.5

    aint = a.to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

# make an RLWE encryption of message
def encrypt_to_fft(m, sfft):
    ct = torch.stack([errgen(stddev, N), uniform(N, Q)])
    ctfft = negacyclic_fft(ct, N, Q)

    ctfft[0] += -ctfft[1]*sfft + negacyclic_fft(m, N, Q)

    return ctfft

def normalize(v, logQ):
    # same as follows but no branch
    """
    if v > Q//2:
        v -= Q
    """
    # vmod Q when Q is a power-of-two
    Q = (1 << logQ)
    v &= Q-1
    # get msb
    msb = (v & Q//2) >> (logQ - 1)
    v -= (Q) * msb
    return v

def decrypt_from_fft(ctfft, sfft):
    assert len(ctfft.size()) == 2
    # normalization is optional
    return normalize(negacyclic_ifft(ctfft[0] + ctfft[1]*sfft, N, Q), logQ)
    # return negacyclic_ifft(ctfft[0] + ctfft[1]*sfft, N, Q)

# we will use B-ary decomposition, i.e., cut digits by logB bits
d = 4
logB = 6

assert d * logB < logQ

decomp_shift = logQ - logB*torch.arange(d,0,-1).view(d,1)
mask = (1 << logB) - 1

gvector = 1 << decomp_shift

def decompose(a):
    
    assert len(a.size()) <= 2
    # for RLWE'
    if len(a.size()) == 1:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1)) & mask
        return res
    # for RGSW
    elif len(a.size()) == 2:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1, 1)) & mask
        return res

msbmask = 0
for i in decomp_shift:
    msbmask += (1<<(i+logB-1))

bin(msbmask)[2:]

# about twice heavier than unsigned decomposition
# it returns value -B/2 <= * <= B/2, not < B/2, but okay
def signed_decompose(a):
    # carry
    da = decompose(a + (a & msbmask))
    # -B
    da -= decompose((a & msbmask))
    return da

def encrypt_rgsw_fft(z, skfft):
    # RGSW has a dimension of d, 2, 2, N
    rgsw = torch.zeros(d, 2, 2, N, dtype=torch.int32)

    # generate the 'a' part
    # INSECURE: to be fixed later
    rgsw[:, :, 1, :] = torch.randint(Q, size = (d, 2 , N), dtype= torch.int32)

    # add error on b
    # INSECURE: to be fixed later
    rgsw[:, :, 0, :] = torch.round(stddev * torch.randn(size = (d, 2, N)))

    # following is equal to rgsw %= Q, but a faster version
    rgsw &= (Q-1)
    rgsw = normalize(rgsw, logQ)

    # do fft for easy a*s
    rgswfft = negacyclic_fft(rgsw, N, Q)

    # now b = -a*sk + e
    rgswfft[:, :, 0, :] -= rgswfft[:, :, 1, :] * skfft.view(1, 1, N//2)

    # encrypt (z, z*sk) multiplied by g
    gzfft = negacyclic_fft(gvector * z, N, Q)
    rgswfft[:, 0, 0, :] += gzfft
    rgswfft[:, 1, 1, :] += gzfft

    return rgswfft

def rgswmult(ctfft, rgswfft):
    ct = negacyclic_ifft(ctfft, N, Q)
    dct = signed_decompose(ct)
    multfft = negacyclic_fft(dct, N, Q).view(d, 2, 1, N//2) * rgswfft
    
    return torch.sum(multfft, dim = (0,1))

In [534]:
def encryptLWE(message, dim, modulus, key):
    ct = uniform(dim + 1, modulus)

    ct[0] = 0

    ct[0] = message * modulus//4 - torch.dot(ct[1:], key)
    ct[0] += errgen(stddev, 1)
    ct &= modulus -1

    return ct

def decryptLWE(ct, sk, modulus):
    m_dec = torch.dot(ct, torch.cat((torch.ones(1, dtype=torch.int32), sk)))
    m_dec %= modulus

    m_dec = m_dec.to(torch.float)
    m_dec /= modulus/4.
    m_dec = torch.round(m_dec)
    return m_dec.to(torch.int)%4

In [535]:
def extract(ctRLWE):
    beta = ctRLWE[0][0]

    alpha = ctRLWE[1][:]
    alpha[1:] = -alpha[1:].flip(dims = [0])

    return torch.cat((beta.unsqueeze(0), alpha))

## 5.1 CMux gate

In each iteration, we need to multiply $RGSW(X^{\alpha_i s_i})$, where $\alpha_i$ is public and $s_i$ is given in encrypted from.
The variants: AP/DM, GINX/CGGI, and LMKCDEY differ in generation of encrypted $X^{\alpha_i s_i}$.

In GINX variant blind rotation for *binary secrets*, we use the following CMux gate as subprocess.
$$
X^{\alpha_i s_i} = (1-s_i) + X^{\alpha_i} \cdot s_i
$$
There are only two possible cases = $s_i$ is $0$ or $1$ - thus, we can easily see that the equation holds in both cases.

1. $s_i = 0$
$$
X^{\alpha_i s_i} = (1-s_i) + X^{\alpha_i} \cdot s_i \\
    = (1-0) + 0\\
    = X^0 
$$
2. $s_i = 1$
$$
X^{\alpha_i s_i} = (1-1) + X^{\alpha_i} \cdot 1 \\
    = (1-1) +  X^{\alpha_i}\\
    = X^{\alpha_i}
$$

Hence, we calculate the following in each iteration
$$
ACC \leftarrow ACC \circledast ((1-RGSW(s_i)) + X^{\alpha_i} \cdot RGSW(s_i)),
$$
which can be computed efficiently with
$$
ACC \leftarrow ACC  + (X^{\alpha_i} - 1 ) \cdot ACC\circledast RGSW(s_i).
$$

We implement the GINX accumulation in the following codes.

In [536]:
s = 1
skN = keygen(N)
skNfft = negacyclic_fft(skN, N, Q)

sPoly = torch.zeros([N], dtype=torch.int32)
sPoly[0] = s

rgswKey = encrypt_rgsw_fft(sPoly, skNfft)

In [537]:
# ACC is transparent encryption of (0,1,2, ..., N)
ACC = torch.zeros([2,N], dtype=torch.int32)
for i in range(N):
    ACC[0][i] = (i%10) * 1000000

ACCfft = negacyclic_fft(ACC, N, Q)

ACC

tensor([[      0, 1000000, 2000000,  ..., 1000000, 2000000, 3000000],
        [      0,       0,       0,  ...,       0,       0,       0]],
       dtype=torch.int32)

In [538]:
alpha = 3
alphaPoly = torch.zeros([N], dtype=torch.int32)
alphaPoly[0] = -1
alphaPoly[alpha] = 1

alphaPolyfft = negacyclic_fft(alphaPoly, N, Q)

In [539]:
# Accumulation
ACCfft += alphaPolyfft * rgswmult(ACCfft, rgswKey)

In [540]:
# similar to (*, *, *, 0, 1, 2, 3, 4, ...) = m * X^{alpha s}
decrypt_from_fft(ACCfft, skNfft)[:10]/1000000

tensor([-1.0021e+00, -2.0053e+00, -2.9950e+00, -2.5510e-03,  1.0061e+00,
         1.9955e+00,  3.0043e+00,  3.9975e+00,  5.0053e+00,  5.9951e+00])

Now we do blind rotation

Generate blind rotation keys

In [541]:
sk = keygen(n)

zero_poly = torch.zeros([N], dtype=torch.int32)

one_poly = torch.zeros([N], dtype=torch.int32)
one_poly[0] = 1

brk = [None]*n
for i in range(n):
    if sk[i] == 0:
        brk[i] = encrypt_rgsw_fft(zero_poly, skNfft)
    else:
        brk[i] = encrypt_rgsw_fft(one_poly, skNfft)

Precompute alphapolys in FFT form

In [542]:
alphapoly = []
for i in range(q):
    poly = torch.zeros([N], dtype=torch.int32)
    poly[0] = -1
    if i < N:
        poly[i] += 1
    else:
        poly[i-N] += -1
    alphapoly.append(negacyclic_fft(poly, N, Q))

First, we generate $f$ for NAND gate.
$[-q/8, 3q/8)$ is mapped to $q/8$, and $[3q/8, 7q/8)$ is mapped to $-q/8$

NOTE: $f$ depends on the binary gate we want to perform.

In [543]:
def nand_map(i):
    i += 2*N 
    i %= 2*N
    if 3*(q>>3) <= i < 7*(q>>3): # i \in [3q/8, 7q/8)
        return -(Q >> 3)
    else: # i \in [-q/8, 3q/8)
        return Q >> 3 

f_nand = torch.zeros([N], dtype=torch.int32)

for i in range(N):
    f_nand[i] = nand_map(-i)

Two LWE encryptions are required

In [544]:
m0 = 0
m1 = 0

ct0 = encryptLWE(m0, n, q, sk)
ct1 = encryptLWE(m1, n, q, sk)

ctsum = (ct0 + ct1) & (q-1)

`ctsum` encrypts `m0 + m1`

In [545]:
decryptLWE(ctsum, sk, q)

tensor(0, dtype=torch.int32)

Initialize `ACC` $= RLWE(f \cdot X^\beta)$.

In [546]:
acc = torch.zeros([2,N], dtype=torch.int32)
acc[0] = f_nand

accfft = negacyclic_fft(acc, N, Q)

beta = ctsum[0]
xbeta = torch.zeros([N], dtype=torch.int32)

if beta < N:
    xbeta[beta] = 1
else:
    xbeta[beta - N] = -1

accfft *= negacyclic_fft(xbeta, N, Q)

Do blind rotation

In [547]:
# for debug
accfft_cpy = accfft.clone()
alpha = ctsum[1:]
for i in range(n):
    ai = alpha[i]
    accfft += alphapoly[ai] * rgswmult(accfft, brk[i])
    accfft_cpy += alphapoly[ai] * (accfft_cpy * sk[i])

    print(i, int(sk[i]), torch.min(decrypt_from_fft(accfft - accfft_cpy, skNfft)/Q), torch.max(decrypt_from_fft(accfft - accfft_cpy, skNfft)/Q))

0 1 tensor(-9.0972e-06) tensor(7.1526e-06)
1 1 tensor(-0.4995) tensor(0.4995)
2 0 tensor(-0.4995) tensor(0.4995)
3 1 tensor(-0.4987) tensor(0.4990)
4 1 tensor(-0.4969) tensor(0.4997)
5 0 tensor(-0.4969) tensor(0.4997)
6 0 tensor(-0.4969) tensor(0.4997)
7 0 tensor(-0.4969) tensor(0.4997)
8 0 tensor(-0.4969) tensor(0.4997)
9 1 tensor(-0.4998) tensor(0.4998)
10 1 tensor(-0.4995) tensor(0.4989)
11 0 tensor(-0.4995) tensor(0.4989)
12 1 tensor(-0.4975) tensor(0.4994)
13 1 tensor(-0.4998) tensor(0.4992)
14 1 tensor(-0.4984) tensor(0.4993)
15 1 tensor(-0.4987) tensor(0.4984)
16 0 tensor(-0.4987) tensor(0.4984)
17 1 tensor(-0.4986) tensor(0.4985)
18 1 tensor(-0.4969) tensor(0.4994)
19 0 tensor(-0.4969) tensor(0.4994)
20 0 tensor(-0.4969) tensor(0.4994)
21 1 tensor(-0.4995) tensor(0.4995)
22 1 tensor(-0.4997) tensor(0.4997)
23 1 tensor(-0.4973) tensor(0.4996)
24 1 tensor(-0.4999) tensor(0.5000)
25 0 tensor(-0.4999) tensor(0.5000)
26 0 tensor(-0.4999) tensor(0.5000)
27 1 tensor(-0.5000) tensor(0.

```
alpha = ctsum[1:]
for i in range(n):
    ai = alpha[i]
    accfft += alphapoly[ai] * rgswmult(accfft, brk[i])
```

```
alpha = ctsum[1:]
for i in range(n):
    ai = alpha[i]
    accfft += alphapoly[ai] * (accfft * sk[i])
```

In [548]:
acc = negacyclic_ifft(accfft, N, Q)
accLWE = extract(acc)
accLWE[0] += (Q >> 3)
accLWE &= Q - 1

In [549]:
# Calculate NAND output
print(f"NAND output: {int(not (m0 and m1))}")
print(f"encrypted result: {decryptLWE(accLWE, skN, Q)}")

NAND output: 1
encrypted result: 1


In [550]:
# What we want from blind rotation
sa = torch.dot(ctsum[1:], sk) & (q-1)

xsa = torch.zeros([N])
if sa < N:
    xsa[sa] = 1
else:
    xsa[sa - N] = -1

accfft *= negacyclic_fft(xsa, N, Q)

In [551]:
acc = negacyclic_ifft(accfft, N, Q)
accLWE = extract(acc)
accLWE[0] += (Q >> 3)
accLWE &= Q - 1

In [552]:
# Calculate NAND output
print(f"NAND output: {int(not (m0 and m1))}")
print(f"encrypted result: {decryptLWE(accLWE, skN, Q)}")

NAND output: 1
encrypted result: 2


Following codes are what we actually do in blind rotation (explained in unencrypted way).

In [553]:
testacc = torch.arange(N)
testaccfft = negacyclic_fft(testacc, N, Q)

alpha = ctsum[1:]
for i in range(n):
    ai = alpha[i]
    testaccfft += alphapoly[ai] * (testaccfft * sk[i])

testaccres = negacyclic_ifft(testaccfft, N, Q)

In [554]:
testaccres &= (2*N-1)
testaccres

tensor([ 865,  866,  867,  ..., 1186, 1185, 1184], dtype=torch.int32)

In [555]:
# test
beta = ctsum[0]
alpha = ctsum[1:]
sa = torch.dot(alpha, sk)

(sa) & (q-1)

tensor(1182, dtype=torch.int32)

In [556]:
if (sa) & (q-1) < N:
    print(testaccres[(sa) & (q-1):])